In [1]:
import cv2  # OpenCVライブラリをインポート
import tensorflow as tf  # TensorFlowライブラリをインポート
from keras.utils import to_categorical  # ラベルをone-hotエンコードするための関数をインポート
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau  # 学習のコールバックをインポート
import numpy as np  # 数値計算ライブラリをインポート
from sklearn.model_selection import train_test_split  # データをトレーニングセットとテストセットに分割する関数をインポート
from sklearn.preprocessing import MinMaxScaler  # データの正規化を行うための関数をインポート
from PIL import Image  # 画像処理ライブラリをインポート
import glob  # ファイルパスのパターンマッチングを行うためのライブラリをインポート
import matplotlib.pyplot as plt  # データの可視化を行うライブラリをインポート
import os  # OS操作を行うライブラリをインポート
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc, roc_curve, f1_score  # モデル評価指標をインポート
from keras.models import Sequential, Model  # Kerasのモデル作成に必要なクラスをインポート
from keras.layers import Input, Flatten, Dense, Dropout, Conv2D, MaxPooling2D, Activation, BatchNormalization  # Kerasのレイヤーをインポート
from keras import optimizers  # Kerasのオプティマイザをインポート
from keras.preprocessing import image  # 画像データの前処理を行うための関数をインポート
import random  # 乱数生成ライブラリをインポート
import csv  # CSV操作ライブラリをインポート
import scipy

# DEBUGフラグを設定
DEBUG = False

# 画像を正方形にする関数
def square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        result = pil_img
        return result
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), (background_color))
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), (background_color))
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

# PIL画像をOpenCV画像に変換する関数
def pil2opencv(in_image):
    out_image = np.array(in_image, dtype=np.uint8)
    return out_image

# OpenCV画像をPIL画像に変換する関数
def opencv2pil(in_image):
    new_image = in_image.copy()
    new_image = Image.fromarray(new_image)
    return new_image

# ハフ変換を用いて画像中の円を検出する関数
def hough(img):
    img_median = cv2.medianBlur(img, 15)
    circles = cv2.HoughCircles(img_median, cv2.HOUGH_GRADIENT, 1.5, 100,
                               param1=150, param2=100, minRadius=50, maxRadius=300)
    circles = np.uint16(np.around(circles))
    img_edges_hough = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    for i in circles[0, :]:
        cv2.circle(img_edges_hough, (i[0], i[1]), i[2], (0, 255, 0), 2)
        cv2.circle(img_edges_hough, (i[0], i[1]), 2, (0, 0, 255), 3)
        HARD_LIMIT_DIM = 288
        biggestCircle = [0, 0, 0]
        for i in circles[0, :]:
            z = i[2]
            if (z > biggestCircle[2]) and (z < HARD_LIMIT_DIM):
                biggestCircle = i
        cv2.circle(img_edges_hough, (biggestCircle[0], biggestCircle[1]), biggestCircle[2], (255, 0, 0), 10)
        cv2.circle(img_edges_hough, (biggestCircle[0], biggestCircle[1]), 2, (255, 0, 0), 10)
        outerRad = int(biggestCircle[2])
        shape = img_edges_hough.shape[1::-1]
        outerCircle = np.zeros((shape[1], shape[0]), dtype=np.uint8)
        cv2.circle(outerCircle, center=(biggestCircle[0], biggestCircle[1]), radius=outerRad, color=255, thickness=-1)
        ori[outerCircle == 0] = [255]
    return ori

# グレースケール画像をRGB画像に変換する関数
def gray_to_rgb(X):
    X_transpose = np.array(X.transpose(0, 1, 2, 3))
    ret = np.empty((X.shape[0], 288, 288, 3), dtype=np.float32)
    ret[:, :, :, 0] = X_transpose[:, :, :, 0]
    ret[:, :, :, 1] = X_transpose[:, :, :, 0]
    ret[:, :, :, 2] = X_transpose[:, :, :, 0]
    return ret.transpose(0, 1, 2, 3)

# データセットの設定
folder = ['Image']
Class = ['Retry', 'Pass']
image_size_x = 288
image_size_y = 216
T_value = 2.082677039
ON_error = 0.07
ON_line_upper = (1 + ON_error) * T_value
ON_line_lower = (1 - ON_error) * T_value

x_train = []
y_train = []
x_val = []
y_val = []
x_test = []
y_test = []

# 傾きデータを読み込む
slope = np.loadtxt('ORR.csv')
print(slope)

# 各フォルダ内のファイルを処理
for index, name in enumerate(folder):
    dir = name
    files = glob.glob(dir + "\*.jpg")
    print(files)

    random_state = 0
    fx_train, fx_test, fy_train, fy_test = train_test_split(files, slope, test_size=0.3, random_state=random_state)
    fx_train, fx_val, fy_train, fy_val = train_test_split(fx_train, fy_train, test_size=0.3, random_state=random_state)

    for i, file in enumerate(fx_train):
        ori = Image.open(file).convert("L")
        ori = ori.resize((image_size_x, image_size_y))
        ori = square(ori, 255)
        ori = pil2opencv(ori)
        hough(ori)
        ori = opencv2pil(ori)

        data = np.asarray(ori)
        x_train.append(data)

        if fy_train[i] < ON_line_lower:
            y_train.append(0)
        elif fy_train[i] > ON_line_upper:
            y_train.append(0)
        else:
            y_train.append(1)

        params = {'rotation_range': 180, 'width_shift_range': 0.1, 'height_shift_range': 0.1, 'vertical_flip': True, 'horizontal_flip': True, 'shear_range': 2}
        datagen = image.ImageDataGenerator(**params)
        x = data[np.newaxis]
        x = x[:, :, :, np.newaxis]
        gen = datagen.flow(x, batch_size=1)

        for n in range(9):
            batches = next(gen)
            aug = np.squeeze(batches)
            x_train.append(aug)
            if fy_train[i] < ON_line_lower:
                y_train.append(0)
            elif fy_train[i] > ON_line_upper:
                y_train.append(0)
            else:
                y_train.append(1)

            if DEBUG:
                gen_img = batches[0].astype(np.uint8)
                plt.subplot(3, 3, n + 1)
                plt.imshow(gen_img)
                plt.axis('off')
                plt.gray()
                plt.tight_layout()
        plt.show()

    for i, file in enumerate(fx_test):
        ori = Image.open(file).convert("L")
        ori = ori.resize((image_size_x, image_size_y))
        ori = square(ori, 255)
        ori = pil2opencv(ori)
        hough(ori)
        ori = opencv2pil(ori)
        data = np.asarray(ori)
        x_test.append(data)

        if fy_test[i] < ON_line_lower:
            y_test.append(0)
        elif fy_test[i] > ON_line_upper:
            y_test.append(0)
        else:
            y_test.append(1)

    for i, file in enumerate(fx_val):
        ori = Image.open(file).convert("L")
        ori = ori.resize((image_size_x, image_size_y))
        ori = square(ori, 255)
        ori = pil2opencv(ori)
        hough(ori)
        ori = opencv2pil(ori)
        data = np.asarray(ori)
        x_val.append(data)

        if fy_val[i] < ON_line_lower:
            y_val.append(0)
        elif fy_val[i] > ON_line_upper:
            y_val.append(0)
        else:
            y_val.append(1)

# データをnumpy配列に変換
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_val = np.array(x_val)
y_val = np.array(y_val)

# データの型を変更し、正規化
x_train = x_train.astype('float32')
x_train = x_train / 255
x_test = x_test.astype('float32')
x_test = x_test / 255
x_val = x_val.astype('float32')
x_val = x_val / 255

# ラベルをone-hotエンコード
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)
y_val = to_categorical(y_val, 2)

# データの形状を変更
train_newarray = (x_train.shape[0], x_train.shape[1], x_train.shape[1], 1)
x_train = np.reshape(x_train, train_newarray)

test_newarray = (x_test.shape[0], x_test.shape[1], x_test.shape[1], 1)
x_test = np.reshape(x_test, test_newarray)

valid_newarray = (x_val.shape[0], x_val.shape[1], x_val.shape[1], 1)
x_val = np.reshape(x_val, valid_newarray)

print("x_train", x_train.shape)
print("x_test", x_test.shape)
print("x_val", x_val.shape)
print("y_train", y_train.shape)
print("y_test", y_test.shape)
print("y_val", y_val.shape)

# Adamオプティマイザの設定
opt = optimizers.Adam(lr=0.05)

# モデルを定義し、トレーニングを行う関数
def model_train(x_train, y_train, x_val, y_val):
    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding="same", input_shape=(x_train.shape[1], x_train.shape[2], 1)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(Conv2D(32, (3, 3), padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Conv2D(256, (3, 3), padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(Conv2D(256, (3, 3), padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Flatten())

    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(Dense(16))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=["accuracy"]
                  )
    print("x_train", x_train.shape)
    print("x_test", x_test.shape)
    print("x_val", x_val.shape)
    print("y_train", y_train.shape)
    print("y_test", y_test.shape)
    print("y_val", y_val.shape)

    early_stopping = [EarlyStopping(patience=20, verbose=1)]
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=10, min_lr=0.0001)

    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=32, epochs=100, shuffle=True, callbacks=[reduce_lr]) 

    model.save_weights('result/cnn_weights.h5')
    model.save('result/cnn_model_weight.h5')

    fig1 = plt.figure()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    fig1.savefig("Accuracy.jpg")

    fig2 = plt.figure()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show
    fig2.savefig("Loss.jpg")

    fig3 = plt.figure()
    plt.title("lr")
    plt.plot(range(len(history.history["lr"])), history.history["lr"])
    plt.show()
    fig3.savefig("lr.jpg")

    return model

# 精度を評価する関数
def acc(y_label, y_pred):
    return np.mean(np.abs(y_label - y_pred) < 0.1)

# モデルを評価する関数
def evaluate(model, x_test, y_test):
    scores = model.evaluate(x_test, y_test, verbose=1)
    print("Test Loss: ", scores[0])
    print("Test accuracy: ", scores[1])

# データをシャッフル
p = np.random.permutation(len(x_train))
x_train, y_train = x_train[p], y_train[p]
u = np.random.permutation(len(x_val))
x_val, y_val = x_val[u], y_val[u]
v = np.random.permutation(len(x_test))
x_test, y_test = x_test[v], y_test[v]

# モデルのトレーニング
model = model_train(x_train, y_train, x_val, y_val)

# モデルの評価
evaluate(model, x_test, y_test)

# 予測を行う
y_pred_train = model.predict(x_train)
y_pred_val = model.predict(x_val)
y_pred_test = model.predict(x_test)
print("pred", y_pred_test)
print("val", y_val)
print("test", y_test)

print("train")
predicted = np.argmax(y_pred_train, 1)
test = np.argmax(y_train, 1)
print(classification_report(test, predicted, target_names=["Retry", "Pass"]))
print("val")
predicted = np.argmax(y_pred_val, 1)
test = np.argmax(y_val, 1)
print(classification_report(test, predicted, target_names=["Retry", "Pass"]))
print("test")
predicted = np.argmax(y_pred_test, 1)
test = np.argmax(y_test, 1)
print(classification_report(test, predicted, target_names=["Retry", "Pass"]))

# 結果をプロット
plt.scatter(y_train, y_pred_train, s=20, c="blue")
plt.scatter(y_val, y_pred_val, s=20, c="green")
plt.scatter(y_test, y_pred_test, s=20, c="red")
plt.legend(['Train', 'Validation', 'Test'])
plt.ylabel('Predict')
plt.xlabel('Test')
plt.xlim(-1, 2)
plt.ylim(-1, 2)
plt.grid()
plt.show()

[2.29028491 2.37422574 2.38904934 2.26560008 2.26694727 2.18397595
 2.15561501 2.17686895 2.15457743 2.20149347 2.19594278 2.262832
 2.37422466 2.23183499 2.24438701 2.17105662 2.21018403 2.22132743
 2.14597794 2.19015642 2.27184333 2.23645657 2.24687036 2.17921941
 2.19442588 2.16961805 2.2841028  2.11835546 2.20366549 2.1565321
 2.18574435 2.24814194 2.21406856 2.26405073 2.16682921 2.26310857
 2.29836411 2.19923401 2.31830456 2.2077608  2.17511552 2.2204661
 2.18832329 2.18414369 2.21486593 2.28435344 2.19704636 2.17546346
 2.25383356 2.23934172 2.22145267 2.22145267 2.26290877 2.26907432
 2.23280372 2.11419868 2.17683479 2.12537728 2.19872149 2.1458813
 2.20341745 2.10861506 2.16111438 2.11291182 2.16974036 2.20793512
 2.20780583 2.33853778 2.26878525 2.32208789 2.28842782 2.3194449
 2.27886566 2.23635925 2.2591678  2.35209424 2.3315693  2.36137102
 2.2997913  2.38702602 2.27945011 2.23190039 2.32496941 2.32702152
 2.28547528]
['Image\\1.jpg', 'Image\\10.jpg', 'Image\\11.jpg', 'Ima

x_train (410, 288, 288, 1)
x_test (26, 288, 288, 1)
x_val (18, 288, 288, 1)
y_train (410, 2)
y_test (26, 2)
y_val (18, 2)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 288, 288, 16)      160       
                                                                 
 activation (Activation)     (None, 288, 288, 16)      0         
                                                                 
 batch_normalization (Batch  (None, 288, 288, 16)      64        
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 288, 288, 32)      4640      
                                                                 
 activation_1 (Activation)   (None, 288, 288, 32)      0         
                                                                 


KeyboardInterrupt: 